In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset
from dual_model import DualModel, DualModelFtype
from dual_train import validate

In [2]:
test_dir_1 = '/home/hank/Datasets/QUASAR/small'
testset_1 = QUASARDataset(test_dir_1,num_graphs=100,remove_self_loops=True)
test_dir_2 = '/home/hank/Datasets/QUASAR/small-test'
testset_2 = QUASARDataset(test_dir_2,num_graphs=100,remove_self_loops=True)
test_dir_3 = '/home/hank/Datasets/QUASAR/N40-100'
testset_3 = QUASARDataset(test_dir_3,num_graphs=100,remove_self_loops=True)
test_dir_4 = '/home/hank/Datasets/QUASAR/N50-100'
testset_4 = QUASARDataset(test_dir_4,num_graphs=100,remove_self_loops=True)
train_dir = '/home/hank/Datasets/QUASAR/N30-1000'
trainset = QUASARDataset(train_dir,num_graphs=1000,remove_self_loops=True)
val_dir = '/home/hank/Datasets/QUASAR/N30-100'
valset = QUASARDataset(val_dir,num_graphs=100,remove_self_loops=True)
DEVICE = torch.device('cpu')

Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.
Data graph type: 1.


In [3]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 64
GNN_OUT_DIM = GNN_HIDDEN_DIM
GNN_LAYER = 7
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
NUM_EPOCHES = 1000
DROPOUT = 0.2
MLP_LAYER = 2
RESIDUAL = True
BATCHNORM = True
model = DualModelFtype(
        node_feature_mode=NODE_MODE,
        gnn_type=GNN_TYPE,
        mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
        dual_node_mlp_hidden_dim=GNN_OUT_DIM,dual_node_mlp_output_dim=10,
        node_mlp_num_layers=MLP_LAYER,
        dual_edge_mlp_hidden_dim=GNN_OUT_DIM,dual_edge_mlp_output_dim=6,
        edge_mlp_num_layers=MLP_LAYER,
        dropout_rate=DROPOUT,
        relu_slope=0.1,
        residual=RESIDUAL,
        batchnorm=BATCHNORM)
model.load_state_dict(torch.load('./models/dual_model_N30-1000_SAGE_7_64_2_True_True_201.pth'))
model.double()

BATCH_SIZE = 5
train_acc = validate(model,trainset,DEVICE,batch_size=BATCH_SIZE)
val_acc = validate(model,valset,DEVICE,batch_size=BATCH_SIZE)
test_acc_1 = validate(model,testset_1,DEVICE,batch_size=BATCH_SIZE)
test_acc_2 = validate(model,testset_2,DEVICE,batch_size=BATCH_SIZE)
test_acc_3 = validate(model,testset_3,DEVICE,batch_size=BATCH_SIZE)
test_acc_4 = validate(model,testset_4,DEVICE,batch_size=BATCH_SIZE)
print('Train acc: {:.4f}.'.format(train_acc))
print('Val acc: {:.4f}.'.format(val_acc))
print('Test acc 1: {:.4f}.'.format(test_acc_1))
print('Test acc 2: {:.4f}.'.format(test_acc_2))
print('Test acc 3: {:.4f}.'.format(test_acc_3))
print('Test acc 4: {:.4f}.'.format(test_acc_4))

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE. Residual: True. BatchNorm: True.


/home/hank/miniconda3/envs/gnnsdp/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Train acc: 0.2818.
Val acc: 0.2711.
Test acc 1: 0.4969.
Test acc 2: 0.4917.
Test acc 3: 0.3608.
Test acc 4: 0.4552.
